# LionAGI - Introduction 11 : Concurrent Branches

In [1]:
import lionagi as li

In [2]:
def multiply(number1:float, number2:float):
    '''
    Perform multiplication on two numbers.

    Args:
        number1: First number to multiply.
        number2: Second number to multiply.
    
    Returns:
        The product of number1 and number2.

    '''
    return number1*number2

tool_mul = li.func_to_tool(multiply)

In [3]:
def add(number1:float, number2:float):
    '''
    Add up two numbers.

    Args:
        number1: First number to add.
        number2: Second number to add.
    
    Returns:
        The sum of number1 and number2.

    '''
    return number1+number2

tool_add = li.func_to_tool(add)

In [4]:
session = li.Session()
session.new_branch('calc_mul')
session.new_branch('calc_add')

In [5]:
session.branches['calc_mul'].register_tools(tool_mul)
session.branches['calc_add'].register_tools(tool_add)

In [6]:
task = "Think step by step, understand the following basic math question and provide parameters for function calling."

# when using respond_mode as json to enforce output format, you need to provide specifying details in instruction
json_format = {"number1": "x", "number2": "y"}

instruct = {"Task": task, "json_format": json_format}

question1 = "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?"
question2 = "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"

context1 = {"Question1": question1, "question2": question2}

question3 = "There are two cards facing up and three facing down, how many cards are there in total?"
context2 = {"Question3": question3}

contexts = {'calc_mul': context1, 'calc_add': context2}

session.default_branch.llmconfig.update({
    "temperature":0.35,
    "tool_choice": "auto", 
    "response_format": {'type':'json_object'}
})

In [7]:
async def calculate(name):
    await session.chat(instruction=instruct, context=contexts[name], to_=name, tools=True)

await li.alcall(['calc_mul', 'calc_add'], calculate)

[None, None]

In [8]:
session.branches['calc_mul'].messages

,node_id,role,sender,timestamp,content
0,80368d3acbc0e1cc61d1300f62051fa5,user,user,2024-02-03 01:16:21.609017,"{""instruction"": {""Task"": ""Think step by step, ..."
1,ab1b46453220d87faeba61772d620a07,assistant,action_request,2024-02-03 01:16:26.511631,"{""action_list"": [{""recipient_name"": ""functions..."
2,93e77da3da185de78f2bb0a40fcd11ab,assistant,action_response,2024-02-03 01:16:26.513599,"{""action_response"": {""function"": ""multiply"", ""..."
3,b763bebb67e8d4f46d8fa1dcfd15d853,assistant,action_response,2024-02-03 01:16:26.515066,"{""action_response"": {""function"": ""multiply"", ""..."


In [9]:
session.branches['calc_add'].messages

,node_id,role,sender,timestamp,content
0,9d16f509b50618273928a4fb0b3f31e3,user,user,2024-02-03 01:16:21.959169,"{""instruction"": {""Task"": ""Think step by step, ..."
1,8179a4b8f2bc7634bf080f72fe07f2e4,assistant,action_request,2024-02-03 01:16:25.211032,"{""action_list"": [{""recipient_name"": ""functions..."
2,6cccc8dca7e038dcba25da7eef022dd9,assistant,action_response,2024-02-03 01:16:25.213402,"{""action_response"": {""function"": ""add"", ""argum..."


# Branch Info Exchange

Each branch can make requests to send messages, tool, service, or llmconfig to another branch.

In [10]:
# suppose branch 'calc_add' request to send its last action response to branch 'calc_mul'
# and branch 'calc_mul' request to senf its tool 'multiply' to branch 'calc_add'

message_request = session.branches['calc_add'].messages.iloc[[-1]]
tool_request = session.branches['calc_mul'].tool_manager.registry['multiply']

session.branches['calc_add'].send('calc_mul', title='messages', package=message_request)
session.branches['calc_mul'].send('calc_add', title='tool', package=tool_request)

`title` is restricted to one of the following type:

- `RequestTitle.MESSAGES` or 'messages'
- `RequestTitle.TOOL` or 'tool'
- `RequestTitle.SERVICE` or 'service'
- `RequestTitle.LLMCONFIG` or 'llmconfig'

After requests are mades, there are three steps to complete the delivery process:
- collect all requests from branches if any
- send all requests to target branches
- branch decide to receive requests

All requests waiting to be collected are in each branch's `pending_outs`, and all requests waiting to be received are in each branch's `pending_ins`.

Each branch can make a receiving decision based on the sender and the request type.

``` python
receive(self, from_name, messages=True, tool=True, service=True, llmconfig=True):

    from_name (str): The name of the sender whose packages are to be processed.
    messages (bool, optional): If True, processes 'messages' requests.
    tool (bool, optional): If True, processes 'tool' requests.
    service (bool, optional): If True, processes 'service' requests.
    llmconfig (bool, optional): If True, processes 'llmconfig' requests.
```

In [11]:
# step by step process
session.collect()
session.send()

session.branches['calc_mul'].receive('calc_add')
session.branches['calc_add'].receive('calc_mul')

# equivalent one step shortcut
# session.collect_send_all(receive_all=True)

In [12]:
session.branches['calc_mul'].messages

,node_id,role,sender,timestamp,content
0,80368d3acbc0e1cc61d1300f62051fa5,user,user,2024-02-03 01:16:21.609017,"{""instruction"": {""Task"": ""Think step by step, ..."
1,ab1b46453220d87faeba61772d620a07,assistant,action_request,2024-02-03 01:16:26.511631,"{""action_list"": [{""recipient_name"": ""functions..."
2,93e77da3da185de78f2bb0a40fcd11ab,assistant,action_response,2024-02-03 01:16:26.513599,"{""action_response"": {""function"": ""multiply"", ""..."
3,b763bebb67e8d4f46d8fa1dcfd15d853,assistant,action_response,2024-02-03 01:16:26.515066,"{""action_response"": {""function"": ""multiply"", ""..."
4,6cccc8dca7e038dcba25da7eef022dd9,assistant,action_response,2024-02-03 01:16:25.213402,"{""action_response"": {""function"": ""add"", ""argum..."


In [13]:
session.branches['calc_add'].tool_manager.registry

{'add': Tool({"node_id":"c4a1abefbf44ca51f08439ab8bbd37c7","metadata":{},"label":null,"related_nodes":[],"content":null,"func":"add","parser":null,"schema_":{"type":"function","function":{"name":"add","description":"Add up two numbers.","parameters":{"type":"object","properties":{"number1":{"type":"number","description":"First number to add."},"number2":{"type":"number","description":"Second number to add."}},"required":["number1","number2"]}}}}),
 'multiply': Tool({"node_id":"7c402d6c64ec91b5c46ddcb4479f524a","metadata":{},"label":null,"related_nodes":[],"content":null,"func":"multiply","parser":null,"schema_":{"type":"function","function":{"name":"multiply","description":"Perform multiplication on two numbers.","parameters":{"type":"object","properties":{"number1":{"type":"number","description":"First number to multiply."},"number2":{"type":"number","description":"Second number to multiply."}},"required":["number1","number2"]}}}})}